### Floyd-Steinberg Dithering

In this homework you are going to implement the Floy-Steinberg dithering algorithm. Dithering, in general, means that we are adding noise to the signal (in our case digital image) in order to perceive it better. In other words, by adding the noise the objective quality will be worse but the subjective quality will be better (i.e. the image will "look" better).

The details of FS dithering can be found in this [wiki](https://en.wikipedia.org/wiki/Floyd%E2%80%93Steinberg_dithering) page. In order to implement the dithering, we will implement the following steps:
* Define colour pallette
* Quantize the image to obtain the baseline and compute the average quantization error
* Implement FS dithering and compute the average quantization error

As always, you are encouraged to use your own images :-)

In [ ]:
import cv2
import math
import numpy as np
from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = [15, 10]

Let's load the image.

In [ ]:
# Load image
img = cv2.imread('...')
# Convert it to RGB
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# Plot it
plt.imshow(img)

Let's start with gray tones first.

In [ ]:
# Black, dark gray, light gray, white
colors = np.array([[0, 0, 0],
                   [64, 64, 64],
                   [192, 192, 192],
                   [255, 255, 255]])

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=16).fit(np.reshape(img, (-1, 3)))
colors = kmeans.cluster_centers_

Using the colour pallette, let's quantize the original image.

In [ ]:
# Cast the image to float
img = img.astype(np.float32)
rows, cols, channels = img.shape
quantized = np.zeros_like(img)

# Apply quantization
for r in range(rows):
    for c in range(cols):
        # Extract the original pixel value
        pixel = img[r, c, :]
        # Find the closest colour from the pallette (using absolute value/Euclidean distance)
        # Note: You may need more than one line of code here        
        diff = colors - pixel
        diff = np.sum(np.abs(diff), axis=1)
        quantized[r, c, :] = colors[np.argmin(diff), :]

In [ ]:
# Show quantized image (don't forget to cast back to uint8)
plt.imshow(quantized.astype(np.uint8))

In [ ]:
# Compute average quantization error
mse = np.mean((img - quantized)**2)
print('PSNR', 10*np.log10(255**2/mse), 'dB')

#### Floyd-Steinberg Dithering
We are now going to implement the FS dithering and compare it to the optimally quantized image we have calculated above.

In [ ]:
# Make a temporal copy of the original image, we will need it for error diffusion
img_tmp = np.copy(img)
dithering = np.zeros_like(img)

for r in range(1, rows-1):
    for c in range(1, cols-1):
        # Extract the original pixel value
        pixel = img_tmp[r, c, :]
        # Find the closest colour from the pallette (using absolute value/Euclidean distance)
        # Note: You may need more than one line of code here        
        diff = colors - pixel        
        diff = np.sum(np.abs(diff), axis=1)
        new_pixel = colors[np.argmin(diff), :]
        
        # Compute quantization error
        quant_error = pixel - new_pixel
        
        # Diffuse the quantization error accroding to the FS diffusion matrix
        # Note: You may need more than one line of code here
        img_tmp[r, c+1] = img_tmp[r, c+1] + 7/16 * quant_error
        img_tmp[r+1, c-1] = img_tmp[r+1, c-1] + 3/16 * quant_error
        img_tmp[r+1, c] = img_tmp[r+1, c] + 5/16 * quant_error
        img_tmp[r+1, c+1] = img_tmp[r+1, c+1] + 1/16 * quant_error        
        
        # Apply dithering
        dithering[r, c] = new_pixel

In [ ]:
# Show quantized image (don't forget to cast back to uint8)
plt.subplot(121), plt.imshow(quantized.astype(np.uint8))
plt.subplot(122), plt.imshow(dithering.astype(np.uint8))

In [ ]:
# Compute average quantization error
mse = np.mean((img - dithering)**2)
print('PSNR', 10*np.log10(255**2/mse), 'dB')


### Questions
* Which image has higher quantization error? Optimally quantized or dithered?
* Which image looks better to you?
* Can you repeat the same process using only two colours: black and white? Show me :-)

### Bonus Points

Repeat the homework using a diffrerent image pallette. For instance, you can use an optimal colour
pallette that we can calculate via k-means algorithm. The following snippet of code will give you the 16
optimal colours for your original image.

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=16).fit(np.reshape(img, (-1, 3)))
colors = kmeans.cluster_centers_
print(colors)

In [ ]:
print(colors.shape)

Apply FS dithering the same way you did before.
* How does the result look like to you?
* What happens if we use 32 colours?
* And what happens if we use 256 colours?